In [1]:

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

2024-01-12 22:09:34.698800: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 22:09:34.720103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 22:09:34.720123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 22:09:34.720702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 22:09:34.724552: I tensorflow/core/platform/cpu_feature_guar

In [2]:
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR','MONTH','DAY'])
#x = x.drop(columns=['Humidity','Precipitation','Wind Speed'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.set_index('DATE', inplace=True)
#x.describe().transpose()
x.head()


,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,15.12,10.68,6.65,3.59
1981-07-02,7.53,13.12,10.68,1.64
1981-07-03,7.32,13.79,37.42,2.43
1981-07-04,10.43,14.40,8.41,2.29
1981-07-05,13.13,15.20,4.13,1.58


In [3]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale = pd.DataFrame(x_scale, columns=x.columns, index=x.index)
x_scale.head()

,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,0.926990,0.385333,0.478536,1.918945
1981-07-02,-1.305674,0.925994,1.044274,-0.352127
1981-07-03,-1.367448,1.074455,4.798080,0.567948
1981-07-04,-0.452614,1.209620,0.725608,0.404897
1981-07-05,0.341614,1.386887,0.124774,-0.422006


In [4]:
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
time_steps = 1

X, y = create_dataset(x_scale, time_steps)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=False)
X_train.shape
#X_val.shape

(12424, 1, 4)

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[time_steps, 4]))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, 'linear'))

cp = tf.keras.callbacks.ModelCheckpoint('model', save_best_only=True)
#model.compile(loss='mse', optimizer='adam', metrics=['rmse'])
model.summary()

2024-01-12 22:09:36.668078: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-12 22:09:36.692057: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-12 22:09:36.695907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                17664     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 18193 (71.07 KB)
Trainable params: 18193 (71.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(loss='Huber', optimizer='adam', metrics=['mae'])

In [8]:
cp = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=cp)

Epoch 1/10


2024-01-12 22:09:37.787547: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


  1/389 [..............................] - ETA: 7:57 - loss: 0.4192 - mae: 0.8062

2024-01-12 22:09:38.185576: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2fb00011d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-12 22:09:38.185593: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-01-12 22:09:38.188834: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705115378.228378   15211 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


389/389 [==============================] - 3s 5ms/step - loss: 0.3617 - mae: 0.7113 - val_loss: 0.3802 - val_mae: 0.7352
Epoch 2/10
389/389 [==============================] - 2s 4ms/step - loss: 0.3601 - mae: 0.7091 - val_loss: 0.3804 - val_mae: 0.7341
Epoch 3/10
389/389 [==============================] - 2s 4ms/step - loss: 0.3598 - mae: 0.7089 - val_loss: 0.3797 - val_mae: 0.7331
Epoch 4/10
389/389 [==============================] - 2s 5ms/step - loss: 0.3597 - mae: 0.7088 - val_loss: 0.3805 - val_mae: 0.7361
Epoch 5/10
389/389 [==============================] - 2s 5ms/step - loss: 0.3597 - mae: 0.7089 - val_loss: 0.3798 - val_mae: 0.7343
Epoch 6/10
389/389 [==============================] - 2s 5ms/step - loss: 0.3596 - mae: 0.7089 - val_loss: 0.3800 - val_mae: 0.7340
Epoch 7/10
389/389 [==============================] - 2s 4ms/step - loss: 0.3596 - mae: 0.7089 - val_loss: 0.3798 - val_mae: 0.7311
Epoch 8/10
389/389 [==============================] - 1s 3ms/step - loss: 0.3596 - mae:

In [9]:
matplotlib.interactive(True)
%matplotlib qt
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [12]:
plt.plot(x['Temperature'])
plt.plot(history.history[])